<a href="https://colab.research.google.com/github/awaiskhan005/DATA-SCIENCE-AND-AI-/blob/main/crypto_price_prediction_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [5]:
# Function to fetch and prepare data for multiple tickers
def fetch_and_prepare_data(tickers, start_date, end_date):
    combined_data = pd.DataFrame()

    for ticker in tickers:
        data = yf.download(ticker, start=start_date, end=end_date)
        data.ffill(inplace=True)  # Forward fill any null values
        data = data[['Close']].rename(columns={'Close': ticker})

        if combined_data.empty:
            combined_data = data
        else:
            combined_data = combined_data.join(data, how='outer')

    combined_data.ffill(inplace=True)  # Forward fill any remaining null values after join
    return combined_data


In [6]:
# Example usage
tickers = ['BTC-USD', 'ETH-USD', 'LTC-USD']
start_date = '2020-01-01'
end_date = '2024-07-28'
data = fetch_and_prepare_data(tickers, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [7]:
# Function to normalize data
def normalize_data(data):
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(data)
    return normalized_data, scaler

# Normalize data
normalized_data, scaler = normalize_data(data)


In [8]:
# Function to create a dataset for LSTM
def create_dataset(data, time_step=1):
    X, Y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step)])
        Y.append(data[i + time_step, 0])  # Predict the first coin (e.g., BTC-USD)
    return np.array(X), np.array(Y)

# Create dataset
time_step = 60
X, Y = create_dataset(normalized_data, time_step)
X = X.reshape(X.shape[0], X.shape[1], len(tickers))  # Reshape for LSTM input


In [9]:
# Function to build LSTM model
def build_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))  # Predict the price of the first coin
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Build and train the model
model = build_model((time_step, len(tickers)))
model.fit(X, Y, batch_size=1, epochs=1)


1609/1609 [==============================] - 55s 32ms/step - loss: 0.0027


In [12]:
def update_model_with_real_time_data(model, scaler, tickers, time_step=60):
    new_data = {}

    for ticker in tickers:
        new_data[ticker] = yf.download(ticker, period='1d', interval='1m')['Close']

    new_data = pd.DataFrame(new_data).ffill().values

    if new_data.shape[0] <= time_step:
        print("Not enough data to update the model.")
        return

    normalized_new_data = scaler.transform(new_data)

    for i in range(len(normalized_new_data) - time_step - 1):
        x = normalized_new_data[i:(i + time_step)]
        x = x.reshape(1, time_step, len(tickers))
        y = normalized_new_data[i + time_step, 0]

        model.train_on_batch(x, np.array([y]))

# Example usage for real-time data update
update_model_with_real_time_data(model, scaler, tickers)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [ ]:
import time
import numpy as np

def make_trading_decision(model, scaler, tickers, time_step=60):
    while True:
        new_data = {}

        for ticker in tickers:
            new_data[ticker] = yf.download(ticker, period='1d', interval='1m')['Close']

        new_data = pd.DataFrame(new_data).ffill().values

        if new_data.shape[0] <= time_step:
            print("Not enough data to make a prediction.")
            continue

        normalized_new_data = scaler.transform(new_data)

        # Predict the next price
        x = normalized_new_data[-time_step:]
        x = x.reshape(1, time_step, len(tickers))
        predicted_price_normalized = model.predict(x)

        # Reshape the predicted price to match scaler input shape
        predicted_price_normalized = predicted_price_normalized.reshape(-1, 1)

        # Inverse transform to get the actual price
        predicted_price = scaler.inverse_transform(np.tile(predicted_price_normalized, (1, len(tickers))))

        # Implement your trading logic here
        print(f"Predicted price for {tickers[0]}: {predicted_price[0][0]}")

        # Sleep for a minute before the next prediction
        time.sleep(60)

# Start the trading bot
make_trading_decision(model, scaler, tickers)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 40ms/step
Predicted price for BTC-USD: 67145.8828125


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 26ms/step
Predicted price for BTC-USD: 67137.6015625


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

...

1/1 [==============================] - 0s 27ms/step
Predicted price for BTC-USD: 67122.375


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1/1 [==============================] - 0s 27ms/step



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Predicted price for BTC-USD: 67120.4921875
